In [ ]:
#@title Datos del estudiante

import requests
!gdown "https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS"

#@markdown Ingrese los datos del estudiante
codigo_estudiante = "123" #@param {type:"string"}
nombre_estudiante = "Jorge" #@param {type:"string"}
grupo = "F3"           #@param {type:"string"}


url = "https://147.185.221.25:28151/registrar-estudiante"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "nombre_estudiante": nombre_estudiante,
    "grupo": grupo
}

res = requests.post(url, json=payload, verify="fullchain.pem")

data = res.json()

print(data.get("mensaje"))


In [ ]:
#@title Librerías
import numpy as np
from scipy.stats import t
from scipy.stats import norm
import inspect

# **Quiz 1**

Se comparan dos productos energéticos, EnergyMax y EnergyPlus, con medias muestrales $\bar{X}_1 = 8.3$ y $\bar{X}_2 = 6.7$ varianzas conocidas $\sigma_1^2 = 1.5$ y $\sigma_2^2 = 1.8$, y tamaños muestrales $n_1 = 40$ y $n_2 = 35$, respectivamente.

Se desea, con un nivel de significancia de $α=0.05$, probar si existe una diferencia significativa entre las medias poblacionales del incremento en la frecuencia cardíaca, así como calcular el error tipo II ($β$) si la diferencia real es $δ=1$.

Además, determine el tamaño de muestra necesario para que $β=0.1$ al detectar una diferencia real de $δ=1$.

In [ ]:
def analisis_z():
    # Datos
    sigma12 = 1.5
    sigma22 = 1.8
    n1 = 40
    n2 = 35
    X1 = 8.3
    X2 = 6.7
    alpha = 0.05

    # Estadístico de prueba
    Z0 = ((X1 - X2) - 0) / np.sqrt((sigma12 / n1) + (sigma22 / n2))

    # Valores críticos
    z_crit_lower = norm.ppf(alpha / 2)
    z_crit_upper = norm.ppf(1 - alpha / 2)

    # Decisión
    if Z0 < z_crit_lower or Z0 > z_crit_upper:
        decision = 'Reject'
    else:
        decision = 'Fail to Reject'

    # P-valor
    Pvalue = 2 * (1 - norm.cdf(abs(Z0)))

    # Intervalo de confianza
    CI_lower = (X1 - X2) - z_crit_upper * np.sqrt((sigma12 / n1) + (sigma22 / n2))
    CI_upper = (X1 - X2) + z_crit_upper * np.sqrt((sigma12 / n1) + (sigma22 / n2))

    # Error tipo II (β) y potencia
    delta = 1
    delta0 = 0
    Beta = norm.cdf(z_crit_upper - ((delta - delta0) / np.sqrt((sigma12 / n1) + (sigma22 / n2)))) \
         - norm.cdf(z_crit_lower - ((delta - delta0) / np.sqrt((sigma12 / n1) + (sigma22 / n2))))
    power = 1 - Beta

    # Tamaño de muestra aproximado
    deltaR = 1
    Beta_d = 0.1
    ZB = norm.ppf(1 - Beta_d)
    n2_aprox = np.ceil(((abs(ZB) + z_crit_upper)**2 * (sigma12 + sigma22)) / (deltaR - delta0)**2)

    # Retornar resultados
    return {
        "Z0": Z0,
        "Valor crítico inferior": z_crit_lower,
        "Valor crítico superior": z_crit_upper,
        "Decisión": decision,
        "P-valor": Pvalue,
        "Límite inferior IC": CI_lower,
        "Límite superior IC": CI_upper,
        "Error tipo II (β)": Beta,
        "Potencia": power,
        "Tamaño de muestra aproximado": n2_aprox
    }

# Ejemplo de uso
resultados = analisis_z()
for k, v in resultados.items():
    print(f"{k}: {v}")


In [ ]:
#@title Calificador Ejercicio 1 - Práctica 6 - F3
resultados = analisis_z()

codigo_funcion = inspect.getsource(analisis_z)

codigo_estudiante = "123"  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 1 - Práctica 6 - F3"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "estadistico_z": resultados["Z0"],
    "valor_critico_inferior": resultados["Valor crítico inferior"],
    "valor_critico_superior": resultados["Valor crítico superior"],
    "decision": resultados["Decisión"],
    "p_valor": resultados["P-valor"],
    "limite_inferior_ic": resultados["Límite inferior IC"],
    "limite_superior_ic": resultados["Límite superior IC"],
    "error_tipo_II": resultados["Error tipo II (β)"],
    "potencia": resultados["Potencia"],
    "tamano_muestra_aprox": resultados["Tamaño de muestra aproximado"],
    "codigo_funcion": codigo_funcion
}

url = "https://147.185.221.25:28151/api/pract/ejer1-prac6-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")


# **Quiz 2**


Se comparan dos medicamentos, A y B, para evaluar su efecto en la reducción de la presión arterial, con medias muestrales $\bar{X}_1 = 10.2$ y $\bar{X}_2 = 8.5$, desviaciones estándar $s_1 = 2.5$ y $s_2 = 1.9$, y tamaños muestrales $n_1 = 15$ y $n_2 = 15$, respectivamente.

Se desea, con un nivel de significancia $\alpha = 0.05$, probar si existe una diferencia significativa entre los efectos promedio de ambos medicamentos, utilizando la varianza combinada y los grados de libertad $df = n_1 + n_2 - 2$; además, se solicita calcular el intervalo de confianza bilateral del 95% y estimar el error tipo II ($\beta$) para una diferencia real de $δ=2$.

Si la potencia mínima es 0.9, asumiendo una diferencia real $\delta = 2$, determinar el tamaño de muestra.


In [ ]:
def medicamento():
    # Datos
    alpha = 0.05
    s1 = 2.5
    s2 = 1.9
    n1 = 15
    n2 = 15
    x1 = 10.2
    x2 = 8.5

    # Varianza combinada
    Sp2 = ((s1**2)*(n1-1) + (s2**2)*(n2-1)) / (n1 + n2 - 2)

    # Estadístico t
    Test_Statistic = (x1 - x2) / (np.sqrt(Sp2) * np.sqrt((1/n1) + (1/n2)))

    # Valores críticos
    t_crit_lower = t.ppf(alpha/2, n1 + n2 - 2)
    t_crit_upper = t.ppf(1 - alpha/2, n1 + n2 - 2)

    # Decisión
    if Test_Statistic > t_crit_upper or Test_Statistic < t_crit_lower:
        decision = 'Reject'
    else:
        decision = 'Fail to Reject'

    # P-valor
    Pvalue = 2 * (1 - t.cdf(abs(Test_Statistic), n1 + n2 - 2))

    # Intervalo de confianza
    CI_lower = round((x1 - x2) + t_crit_lower * np.sqrt(Sp2) * np.sqrt((1/n1) + (1/n2)), 6)
    CI_upper = round((x1 - x2) + t_crit_upper * np.sqrt(Sp2) * np.sqrt((1/n1) + (1/n2)), 6)

    # Potencia y error tipo II
    d = abs(2 / (2 * np.sqrt(Sp2)))
    n = 15
    beta = 0.6
    power = 1 - beta

    # Cálculo de tamaño de muestra
    Power = 0.9
    Beta = 1 - Power
    na = 50
    n2_aprox = np.ceil((na + 1) / 2)

    # Retornar resultados
    return {
        "Estadístico t": Test_Statistic,
        "Valor crítico inferior": t_crit_lower,
        "Valor crítico superior": t_crit_upper,
        "Decisión": decision,
        "P-valor": Pvalue,
        "Límite inferior IC": CI_lower,
        "Límite superior IC": CI_upper,
        "d": d,
        "Error tipo II (β)": beta,
        "Potencia": power,
        "Tamaño de muestra aproximado": n2_aprox
    }

# Ejemplo de uso
resultados = medicamento()
for k, v in resultados.items():
    print(f"{k}: {v}")


In [ ]:
#@title Calificador Ejercicio 2 - Práctica 6 - F3

resultados = medicamento()

codigo_funcion = inspect.getsource(medicamento)

codigo_estudiante = "123"  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 2 - Práctica 6 - F3"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "estadistico_t": resultados["Estadístico t"],
    "valor_critico_inferior": resultados["Valor crítico inferior"],
    "valor_critico_superior": resultados["Valor crítico superior"],
    "decision": resultados["Decisión"],
    "p_valor": resultados["P-valor"],
    "limite_inferior_ic": resultados["Límite inferior IC"],
    "limite_superior_ic": resultados["Límite superior IC"],
    "potencia": resultados["Potencia"],
    "error_tipo_II": resultados["Error tipo II (β)"],
    "d": resultados["d"],
    "tamano_muestra_aprox": resultados["Tamaño de muestra aproximado"],
    "codigo_funcion": codigo_funcion
}

url = "https://147.185.221.25:28151/api/pract/ejer2-prac6-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")
